In [19]:
import pandas as pd
import numpy as np
import math

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [20]:
data = pd.read_csv('data/netflix_titles.csv')
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [21]:
data['description'] = data['description'].fillna('')

In [22]:
# Преобразование признаков TF-IDF
model = TfidfVectorizer(stop_words='english')
feature_matrix = model.fit_transform(data['description'])

In [23]:
feature_matrix.shape

(7787, 17905)

In [24]:
# Определение косинусной близости
# Мы используем здесь linear_kernel(), а не cosine_similarity(), так как в косинусном расстоянии в знаменателе реализуется нормировка векторов, 
# а TF-IDF создаёт уже нормализованные векторы.
cosine_sim = linear_kernel(feature_matrix, feature_matrix)

In [25]:
# Массив фильмов с их индексами
indices = pd.Series(data.index,index=data['title']).drop_duplicates()

In [26]:
def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores = scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return data['title'].iloc[ind_movie]

In [27]:
get_recommendations('Balto')

709                Balto 2: Wolf Quest
7446                           Vroomiz
1338    Chilling Adventures of Sabrina
7388                          Vampires
1770                          Dinotrux
2767                     Hold the Dark
5540                 Shanghai Fortress
4041                             Mercy
2582                       Half & Half
1365        Christmas in the Heartland
Name: title, dtype: object

КОЛЛАБОРАТИВНАЯ ФИЛЬТРАЦИЯ

In [28]:
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic, accuracy
from surprise.model_selection import train_test_split

In [29]:
data = Dataset.load_builtin("ml-100k")
trainset, testset = train_test_split(data, test_size=0.25, random_state=13)

In [30]:
sim_options = {
    'name': 'cosine',
    'user_based': False
}

# memory-basic-подход
knn = KNNBasic(sim_options=sim_options)
knn.fit(trainset)
predictions = knn.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0272


1.0271678039029761

In [31]:
pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)

In [32]:
recom = pred[pred.uid =='849']['iid'].to_list()
recom

['234', '427', '568', '174']

In [33]:
# user-based-подход
sim_options = {
    'name': 'cosine',
    'user_based': True
}

knn = KNNBasic(sim_options=sim_options)
knn.fit(trainset)
predictions = knn.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0175


1.0174852296380237

In [34]:
# model-based-подход
model = SVD()
model.fit(trainset)
predictions = model.test(testset)
accuracy.rmse(predictions)

RMSE: 0.9417


0.9417423610952942

ГИБРИДНЫЕ МОДЕЛИ РЕКОМЕНДАТЕЛЬНЫХ СИСТЕМ

In [35]:
#from lightfm import LightFM
#from lightfm.cross_validation import random_train_test_split
#from lightfm.evaluation import precision_at_k, recall_at_k

ModuleNotFoundError: No module named 'lightfm'

In [36]:
from scipy.sparse import csr_matrix

In [37]:
ratings = pd.read_csv('data/math15/ratings.csv') #поставленные оценки
books = pd.read_csv('data/math15/books.csv') #информация о книгах
tags = pd.read_csv('data/math15/tags.csv') #информация о тегах
book_tags = pd.read_csv('data/math15/book_tags.csv') #книги с тегами 

In [38]:
ratings_matrix = csr_matrix((ratings.rating,(ratings.user_id,ratings.book_id))) 
# передаём в качестве аргументов в функцию выставленный рейтинг (это будут значения матрицы), 
# а также id пользователя и id книги (это будут индексы для строк и столбцов матрицы)

In [41]:
# Теперь нам необходимо составить матрицу с метаданными. В качестве индексов будут выступать id книги и id тега, 
# и если у этой книги есть рассматриваемый тег, то на пересечении соответствующих строки и столбца будет выставлена единица.
meta_matrix  = csr_matrix(([1]*len(book_tags),(book_tags.goodreads_book_id,book_tags.tag_id)))

In [ ]:
#model = LightFM(loss='warp', #определяем функцию потерь
                #random_state=13, #фиксируем случайное разбиение
                #no_components=100) #размерность вектора для представления данных в модели

В качестве функции потерь мы выбрали значение 'warp', хотя, разумеется, это не единственный вариант. В модуле LightFM представлены следующие функции потерь:

'logistic' — логистическая функция. Полезна в случаях, когда есть как положительные, так и отрицательные взаимодействия, например 1 и -1.

'bpr' — байесовский персонализированный рейтинг. Можно применять, когда присутствуют только положительные взаимодействия.

'warp' — парный взвешенный приблизительный ранг. Используется, если необходимо повысить качество именно в верхней части списка рекомендаций.

'warp-kos' — модификация warp.

In [42]:
#train,test = random_train_test_split(ratings_matrix, test_percentage=0.3, random_state=13)

NameError: name 'random_train_test_split' is not defined

In [ ]:
#model = model.fit(train, item_features = meta_matrix)

In [ ]:
#prec_score = precision_at_k(
                     #model,
                     #test,
                     #item_features = meta_matrix).mean() 
#print(prec_score)

DEEP LEARNING

In [43]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

In [45]:
from sklearn.model_selection import train_test_split

In [44]:
df = pd.read_csv('data/math15/ratings.csv')

In [47]:
train,test = train_test_split(df, test_size=0.2, random_state=42)

In [53]:
n_books = df.book_id.nunique()
n_users = df.user_id.nunique()

In [54]:
# Создаём эмбеддинги для книг
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

In [55]:
#Создаём эмбеддинги для пользователей
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

In [56]:
# Соединяем ембеддинги
conc = Concatenate()([book_vec, user_vec])

In [57]:
# Далее начинаем «собирать» нашу нейронную сеть из слоёв. Dense обозначает полносвязный слой. 
# Также мы обозначаем для него количество нейронов и данные, которые идут на вход.
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

In [58]:
# Собираем модель — передаём входные данные для книг и пользователей, а также архитектуру нейронной сети:
model2 = Model([user_input, book_input], out)

In [59]:
# Также нам необходимо задать алгоритм оптимизации и метрику, которую мы будем оптимизировать. 
# В данном случае будем использовать метод adam и среднеквадратичную ошибку
model2.compile(optimizer = 'adam',loss =  'mean_squared_error')

In [61]:
# Обучение модели
# В параметр эпох передаём значение 5: у нас будет реализовано пять эпох — пять обучений нейронной сети. 
# На каждой из эпох обновляются веса для минимизации ошибки.
history = model2.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)

Epoch 1/5
24544/24544 [==============================] - 243s 10ms/step - loss: 0.8007
Epoch 2/5
24544/24544 [==============================] - 182s 7ms/step - loss: 0.6893
Epoch 3/5
24544/24544 [==============================] - 175s 7ms/step - loss: 0.6618
Epoch 4/5
24544/24544 [==============================] - 182s 7ms/step - loss: 0.6373
Epoch 5/5
24544/24544 [==============================] - 185s 8ms/step - loss: 0.6181


In [62]:
model2.evaluate([test.user_id, test.book_id], test.rating)

6136/6136 [==============================] - 9s 1ms/step - loss: 0.7096


0.7096015810966492

In [63]:
# Обычно для улучшения качества модели каким-то образом модифицируют нейронную сеть: дополняют её, увеличивают время обучения. 
# Добавим ещё один полносвязный слой с восемью нейронами после полносвязного слоя с 32 нейронами. 
# Обучим нейронную сеть, реализовав десять эпох:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)
model2.evaluate([test.user_id, test.book_id], test.rating)

Epoch 1/10
24544/24544 [==============================] - 166s 7ms/step - loss: 0.6385
Epoch 2/10
24544/24544 [==============================] - 176s 7ms/step - loss: 0.5910
Epoch 3/10
24544/24544 [==============================] - 191s 8ms/step - loss: 0.5717
Epoch 4/10
24544/24544 [==============================] - 169s 7ms/step - loss: 0.5536
Epoch 5/10
24544/24544 [==============================] - 173s 7ms/step - loss: 0.5375
Epoch 6/10
24544/24544 [==============================] - 144s 6ms/step - loss: 0.5241
Epoch 7/10
24544/24544 [==============================] - 158s 6ms/step - loss: 0.5131
Epoch 8/10
24544/24544 [==============================] - 152s 6ms/step - loss: 0.5036
Epoch 9/10
24544/24544 [==============================] - 137s 6ms/step - loss: 0.4952
Epoch 10/10
6136/6136 [==============================] - 7s 1ms/step - loss: 0.7819


0.781889021396637